In [1]:
import time, os, socket, threading
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import \
ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
import cv2

In [2]:
# chorme_options = webdriver.ChromeOptions()
# chorme_options.add_argument('--headless')
# chorme_options.add_argument('--no-sandbox')
# chorme_options.add_argument('--disable-dev-shm-usage')
# webbrowser를 띄우지 않을때

In [3]:
#search = "coffee" # 검색 키워드
chromedriver_path = './chromedriver/chromedriver' # 드라이버 파일 위치

# Crawling Func

In [4]:
def create_folder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def scroll_down() :
    scroll_count = 0
    print("[scroll_down() : 스크롤 다운 시작]")
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    after_click = False
    
    while True :
        print(f"[스크롤 다운: {scroll_count}]")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        time.sleep(1)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if last_height == new_height :
            if after_click is True :
                break
            else :
                try :
                    more_button = driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input')
                    if more_button.is_displayed():
                        more_button.send_keys(Keys.ENTER)
                        after_click = True
                except NoSuchElementException as e:
                    # scroll button이 없을때
                    print(e)
                    break
        last_height = new_height
        
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count
    
    try:
        img.click()
        driver.implicitly_wait(5) # 5초 기다림
        xpath = '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img'
        src = driver.find_element('xpath', xpath).get_attribute('src')
#         src = driver.find_element(By.CSS_SELECTOR, '.n3VNCb.KAlRDb').get_attribute('src')
        if src.split('.')[-1] == 'png':
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".png")
            print(f"{index+1}/{img_list_length} PNG 이미지 저장")
        else :
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".jpg")
            print(f"{index+1}/{img_list_length} JPG 이미지 저장")

        scraped_count += 1
    except HTTPError as e :
        print(e)
    except Exception as e :
        print(e)
        return
    
def scraping(dir_name, query):
    global scraped_count

    url = f"https://www.google.com/search?q={query}&tbm=isch&ved=2ahUKEwi037baoc76AhVEyIsBHQnWDywQ2-cCegQIABAA&oq=f&gs_lcp=CgNpbWcQARgAMgQIIxAnMgQIIxAnMggIABCABBCxAzIFCAAQgAQyCAgAEIAEELEDMgUIABCABDIICAAQgAQQsQMyBQgAEIAEMgUIABCABDIFCAAQgARQxwJYxwJggxJoAHAAeACAAYYBiAHhAZIBAzEuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=4C5AY7TvMMSQr7wPiay_4AI&bih=969&biw=1920&rlz=1C5CHFA_enKR1026KR1026#imgrc=X6d1rGBwHMhXsM"
    driver.get(url)
    driver.maximize_window()
    

    scroll_down()
    print('[스크롤 완료]')
    driver.execute_script("window.scrollTo(0, 0)")
    
    div = driver.find_element('xpath', '//*[@id="islrg"]/div[1]')
    img_list = div.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
#     img_list = div.find_elements('xpath', '//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    print(len(img_list))
    img_list_length = len(img_list)
    #######################################################################################

    for index, img in enumerate(img_list) :
        try:
            click_and_save(dir_name, index, img, img_list_length)
        except ElementClickInterceptedException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except NoSuchElementException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except ConnectionResetError as e :
            print(e)
            pass
        except URLError as e :
            print(e)
            pass
        except socket.timeout as e :
            print(e)
            pass
        except socket.gaierror as e :
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break
            
    try :
        print("[스크래핑 종료 (성공률: {:.2f})]".format((scraped_count / img_list_length) * 100))
    except ZeroDivisionError as e :
        print(e)
    
    driver.quit()

# Preprocess Func

In [5]:
def resize_and_remove(dir_name, query, filter_size):
    # 일정 해상도 이하이거나 손상된 이미지 제거
    filtered_count = 0
    
    for index, file_name in enumerate(os.listdir(dir_name)):
        try :
            file_path = os.path.join(dir_name, file_name)
            img = cv2.imread(file_path)
            height = img.shape[0]
            width = img.shape[1]
            if height >= width and width > 512 :
                img = cv2.resize(img, (512, int(512/height*width)), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
            elif width > height and height > 512 :
                img = cv2.resize(img, dsize=(int(512/width*height), 512), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
                
        except OSError as e :
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except AttributeError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except TypeError as e:
            print(e)
            print(img.shape)
    print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}]")

In [6]:
query_list = ['flooding', 'flooding road', '강남역 침수','침수', 
              '道路浸水', '車の洪水', # 중국어, 일본어 (도로침수, 차 침수)
             'Überschwemmungen auf der Straße', # 독일어, 도로 침수
             'inondation des routes', # 프랑스어, 도로 침수
              'inundación de coches', # 스페인어, 차 침수
              'flooded', '침수 cctv',] 

In [ ]:
socket.setdefaulttimeout(30)
driver = webdriver.Chrome(chromedriver_path)

scraped_count = 0

path = './data/'
#query = input('검색어 입력: ')
query = 'inundación de coches'
dir_name = path+query
create_folder(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
print('##################### 크롤링 끝 #############################')
resize_and_remove(dir_name, query, 512)

/var/folders/63/8nyxhd_90vg4rctvs_pfm8440000gn/T/ipykernel_56099/158270248.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


[./data/inundación de coches 디렉토리 생성]
[scroll_down() : 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 완료]
878
1/878 JPG 이미지 저장
2/878 JPG 이미지 저장
3/878 JPG 이미지 저장
4/878 JPG 이미지 저장
5/878 JPG 이미지 저장
6/878 JPG 이미지 저장
7/878 JPG 이미지 저장
8/878 JPG 이미지 저장
9/878 JPG 이미지 저장
10/878 JPG 이미지 저장
11/878 JPG 이미지 저장
12/878 JPG 이미지 저장
13/878 JPG 이미지 저장
14/878 JPG 이미지 저장
15/878 JPG 이미지 저장
16/878 JPG 이미지 저장
17/878 JPG 이미지 저장
18/878 JPG 이미지 저장
19/878 JPG 이미지 저장
20/878 JPG 이미지 저장
21/878 JPG 이미지 저장
22/878 JPG 이미지 저장
23/878 JPG 이미지 저장
24/878 JPG 이미지 저장
25/878 JPG 이미지 저장
26/878 JPG 이미지 저장
27/878 JPG 이미지 저장
28/878 JPG 이미지 저장
29/878 JPG 이미지 저장
30/878 JPG 이미지 저장
31/878 JPG 이미지 저장
32/878 JPG 이미지 저장
33/878 JPG 이미지 저장
34/878 JPG 이미지 저장
35/878 JPG 이미지 저장
36/878 JPG 이미지 저장
37/878 JPG 이미지 저장
38/878 JPG 이미지 저장
39/878 JPG 이미지 저장
40/878 JPG 이미지 저장
41/878 JPG 이미지 저장
42/878 JPG 이미지 저장
43/878 JPG 이미지 저장
44/878 JPG 이미지 저장
45/878 JPG 이미지 저장
46/878 JPG 이

429/878 JPG 이미지 저장
430/878 JPG 이미지 저장
431/878 JPG 이미지 저장
432/878 JPG 이미지 저장
433/878 JPG 이미지 저장
434/878 JPG 이미지 저장
435/878 JPG 이미지 저장
436/878 JPG 이미지 저장
437/878 JPG 이미지 저장
438/878 JPG 이미지 저장
439/878 JPG 이미지 저장
440/878 JPG 이미지 저장
441/878 JPG 이미지 저장
442/878 JPG 이미지 저장
443/878 JPG 이미지 저장
444/878 JPG 이미지 저장
445/878 JPG 이미지 저장
446/878 JPG 이미지 저장
447/878 JPG 이미지 저장
448/878 JPG 이미지 저장
449/878 JPG 이미지 저장
450/878 JPG 이미지 저장
451/878 JPG 이미지 저장
452/878 JPG 이미지 저장
453/878 JPG 이미지 저장
454/878 JPG 이미지 저장
455/878 JPG 이미지 저장
456/878 JPG 이미지 저장
457/878 JPG 이미지 저장
458/878 JPG 이미지 저장
459/878 JPG 이미지 저장
460/878 JPG 이미지 저장
461/878 JPG 이미지 저장
462/878 JPG 이미지 저장
463/878 JPG 이미지 저장
464/878 JPG 이미지 저장
465/878 JPG 이미지 저장
466/878 JPG 이미지 저장
467/878 JPG 이미지 저장
468/878 JPG 이미지 저장
469/878 JPG 이미지 저장
470/878 JPG 이미지 저장
471/878 JPG 이미지 저장
472/878 JPG 이미지 저장
473/878 JPG 이미지 저장
474/878 JPG 이미지 저장
475/878 JPG 이미지 저장
476/878 JPG 이미지 저장
477/878 JPG 이미지 저장
478/878 JPG 이미지 저장
479/878 JPG 이미지 저장
480/878 JPG 이미지 저장
481/878 JPG 

In [16]:
num_img = 0
folders = os.listdir(path)

for folder in folders :
    if folder == '.DS_Store' :
        continue
    folder_path = os.path.join(path, folder)
    print('{} : {}'.format(folder, len(os.listdir(folder_path))))
    num_img += len(os.listdir(folder_path))
print('total : ', num_img)

inondation des routes : 765
침수 cctv : 698
車の洪水 : 891
침수 : 831
flooding road : 588
Überschwemmungen auf der Straße : 889
道路浸水 : 835
flooded : 817
flooding : 709
강남역 침수 : 495
inundación de coches : 876
total :  8394
